# Key statements

1. We may view $\ell$-step lookahead minimization as the special case of its one-step counterpart where the lookahead function is the optimal cost function of an $(\ell - 1)$-stage DP problem that starts at $x_{k+1}$ and has a terminal cost $\alpha^{\ell} \tilde{J} (x_{k+\ell})$ after $\ell - 1$ stages.

2. In MPC problems taht involve state constraints, it is necessary to modify the state space $X$ to ensure that the $\ell$-step lookahead minimization has a feasible solution. This leads to the problem of reachability of a target tube. In the context of the off-line training/on-line play conceptual framework of the present paper, reachability issues are ordinarily dealt with off-line, as they tend to involve substantial preliminary target tube calculations. **An alternative and simpler possibility is to replace the state constraints with penalty or barrier functions as part of the cost per stage.**

3. Rollout is a major RL approach. It is closely connected to the MPC design philosophy. An important conceptual point is that rollout consists of a single iteration of the fundamental DP method of policy iteration, whose connection with Newton's method in the context of linear-quadratic problems, and other Markov decision problems is well known.

4. The main difficulty with rollout is that computing the required values of cost function value based on a stable policy online may require time consuming simulation. This is an even greater difficulty for the $\ell$-step lookahead version of rollout, where the required number of base policy values increases exponentially with $\ell$. In this case, approximate versions of rollout may be used, such as _simplified_rollout_, _truncated_rollout_, and _multiagent_rollout_.

5. Implicit in approximation in value space is a conceptual separation between two algorithms:
   * The off-line training algorithm, which designs cost function approximation $\tilde{J}$ and possibly other problem components (such as for example a base policy for rollout, or a target/safety tube of states where the system must stay at all times).
   * The on-line play algorithm, which implements the policy $\tilde{\mu}$ in real-time via one-step or $\ell$-step lookahead minimization.
  
6. The architectures of AlphaZero and MPC are very similar. They both involve optimization over a truncated rolling horizon with a terminal cost approximation.It is presently believed that the principal contributor to their success is long lookahead, which uses an efficient on-line play algorithm that involves various forms of tree pruning. The off-line trained position evaluator and player have also contributed to success, although likely a less extent.

7. (a) The online player of AlphaZero plays much better than its extensively trained off-line player. This is due to the beneficial effect of policy improvement with long lookahead minimization, which corrects for the inevitable imperfections of the NN-trained offline player, and position evaluator/terminal cost approximation.

   (b) The TD-Gammon player that uses long rollout with a policy plays much better than TD-Gammon without rollout. This is due to the beneficial effect of the roll-out, which serves as a substitute for long lookahead minimization. 

9. **IMPORTANT**
   The performance of an off-line trained policy can be greatly improved by on-line approximation in value space, with long lookahead (involving minimization or rollout with the off-line policy, or both), and terminal cost approximation that is obtained off-line. This performance enhancement is often dramatic and is due to a simple fact, which is couched on algorithmic mathematics and is a focal point of the present paper:
   (a) Approximation in value space with one-step lookahead minimization amounts to a step of Newton's method for solving Bellman's equation.
   (b) The starting point for the Newton step is based on the results of off-line training, and can be enhanced by longer lookahead minimization and on-line rollout.

# overview of the Framework

1. One-step lookahead is equivalent to a step of Newton's method for solving the Bellman equation.
2. $\ell$-step lookahead is equivalent to a step of a Newton/SOR (_successive over relaxation_) method, whereby the Newton step is preceded by $\ell - 1$ SOR steps.
3. There is a superlinear relation between the approximation error $\left \| \tilde{J} - J^* \right \|$ and the performance error $\left \| J_{\tilde{\mu}} - J^* \right \|$, owing to the preceding Newton step interpretation. As a result, within the region of convergence of Newton's method, the performance error $\left \| J_{\tilde{\mu}} - J^* \right \|$ is small and often negligible. In particular, the MPC policy $\tilde{\mu}$ is very close to optimal if $\tilde{J}$ lies within the region of superlinear convergence of Newton's method.
4. The region of convergence of Newton's method expands as the length $\ell$ of the lookahead minimization increases. Thus the performance of the MPC policy $\tilde{\mu}$ improves as $\ell$ increases, and is essentially optimal if $\ell$ is sufficiently large regardless of the quality of the _terminal cost approximation_ $\tilde{J}$. Indeed, for finite state and control spaces, discount factor $\alpha < 1$ and a long enough lookahead, it can be shown that $\tilde{\mu}$ is an optimal policy, regardless of the size of the approximation error.
5. The region of stability, i.e., the set of $\tilde{J}$ for which $\tilde{\mu}$ is stable in the sense that $J_{\tilde{\mu}} (x) < \infty$ for all $x \in X$, is closely connected to the region of convergence of Newton's method.
6. The region of stability is also enlarged by increasing the length of the rollout horizon, as long as the base policy that is used for rollout is stable.
7. Rollout with a stable policy $\mu$ guarantees that the lookahead policy $\tilde{\mu}$ is also stable, regardless of the length $\ell $ of lookahead. 

## Truncated Rollout

Truncated rollout makes little sense in linear-quadratic problems where $K_L$ can be easily computed by solving the Riccati equation. However, it is useful in more general problem settings, as it may save significantly in computation, relative to obtaining exactly $J_{\mu}$.

* Lookahead by truncated rollout may be an economic substitute for lookahead by minimization, in the sense that it may achieve a similar performance at significantly reduced computational cost;
* Lookahead by m-step truncated rollout with a stable policy has an increasingly beneficial effect on the stability properties of the lookahead policy, as m increases.


## The Importance of the First Step of Lookahead

The Newton step interpretation of approximation in value space leads to an important insight into the special character of the initial step in $\ell$-step lookahead implementations. In particular, it is only the first step that acts as the Newton step, and needs to be implemented with precision. The subsequent steps consist of a sequence of value iterations only serve to enhance the quality of the starting point of the Newton step. As a result, their precise implementation is not critical. 

This idea suggests that we can simplify the lookahead steps after the first with small performance loss for the multistep lookahead policy. On the other hand, pruning the lookahead tree at the first stage of lookahead, as is often done in Monte Carlo Tree Search, can have a serious detrimental effect on the quality of the MPC policy. 


### How approximation in value space can fail and what to do about it

Assume that $\tilde{J}$ is obtained by training with data in a NN. For the case of one-step lookahead with no truncated rollout, there are three components that determine the approximation error $\tilde{J} - J^*$:

a). The power of the NN architecture, which roughly speaking is a measure of the error that would be obtained if infinite data were available and used optimally to obtain $\tilde{J}$ by training the given NN. 

b). The additional error degradation due to limited availability of training data. 

c). The additional error degradation due to imperfections in the training methodology. 

Thus if the architecture is not powerful enough to bring $\tilde{J} - J^*$ within the region of convergence of Newton's method, approximation in value space with one-step lookahead will likely fail, no matter how much data is collected and how effective the associated training method is. 

In this case, there are two potential practical remedies:

1. Use a more powerful architecture/ neural network for representing $\tilde{J}$, so it can be brought closer to $J^*$.

2. Extend the combined length of the lookahead minimization and truncated rollout in order to bring the effective value of $\tilde{J}$ within the region of convergence of Newton's method.

The first remedy requires a deep NN or transformer, which uses more weights and requires more expensive training. The second remedy requires longer on-line computation and/or simulation, which may run to difficulties because of some practical implementation limits. Parallel computation and sophisticated multistep lookahead methods may help to mitigate these requirements. 

## The treatment of stochastic uncertainty through certainty equivalence (CE)

In the pure but somewhat flawed version of the CE approach, when solving the $\ell$-step lookahead minimization problem, we simply replace all of the uncertain quantities by some fixed nominal values, thus making that problme fully deterministic. Unfortunately, this affects significantly the character of the approximation: when $w_k$ is replaced by a deterministic quantity, the Newton step interpretation of the underlying approximation in value space scheme is lost to a great extent.

Still, we may largely correct this difficulty, while retaining substantial simplification, by using CE after the first stage of the $\ell$-step lookahead. We can do this with a CE scheme whereby at state $x_k$, we replace only the uncertain quantities by deterministic values, while treating the first $w_k$ as a stochastic quantity. This way, we maintain the Newton step character of the approximation in value space scheme. 

## MPC and Adaptive Control

The idea of learning from interaction with the environment is often connected with the idea of exploring the environment to identify its characteristics. In control theory this is often view as part of the system identification methodology, which aims to construct math models of dynamic systems. The system identification process is often combined with the control process to deal with unknown or changing problem parameters in a framework that is sometimes called dual control. This is one of the most challenging areas of stochastic optimal and suboptimal control. 

1. Robust Control
   * Given a controller design that has been obtained assuming a nomial DP problem model, one possibility is to simply ignore changes in problem parameters. We may then try to investigate the performance of the current design for a suitable range of problem parameter values, and ensure that it is adequate for the entire range. THis is sometimes called a robust controller design.
   * A simple time-honored robust/adaptive control approach for continuous-state problems is PID control.
   * PID control is used widely but its range of application is mainly restricted to single-input, single-output continuous-state control systems.
  
2. Dealing with unknown parameters through system identification and reoptimization - Online replanning
   * An ambitious form of suboptimal control is to separate the control process into two phases, a system identification pahse (the unknown parameters are estimated, while the control takes no ccount of the interim results of estimation) and a control phase (the final parameter estimates from the first phase are then used to implement an optimal or suboptimal policy in the second phase)
   * Challenges: it may be hard to recompute an optimal or near-optimal policy on-line, using a newly identified system model. In particular, it may be impossible to use time-consuming methods that involve for example the training of a NN or discrete/integer control constraints.
  
3. Adaptive Control by Rollout
   * The advantage of rollout is that it is simple and reliable. In particular, it does not require a complicated training procedure to revise the current policy, based for example on the use of NN or other approximation architecutres, so no new policy is explicitly computed in response to the parameter changes. Instead the current policy is used as the base policy for rollout, and the available controls at the current state are compared by a one-step or multistep minimization, with cost function approximation provided by the base policy.
   * Overtime the base policy may also be revised. In this case, the rollout policy will be adjusted both in response to the changed current policy and in response to the changing parameters. This is necessary in particular when the constraints of the problem change.

# Conclusion

Connections between the MPC and RL fields are strong, and that the most successful design architectures of the two fileds share important characteristics, which relate to Newton's method. 

A principal theoretical reason for the successes of the two fields is the off-line training/ on-line play synergism that rests upon the mathematical foundations of Newton's method. 

Still the cultures of MPC and RL have different starting points and have grown in different directions. One of the primary reason is the preference for continuous state and control spaces in MPC, which stems from the classical control theory tradition. Thus stability and safety/reachability issues have been of paramount importance in MPC, but they are hardly ever considered in RL. 

The main reason is that stability does not arise mathimatically or practically in the discrete state and control contexts of games, Markovian decision problems, and more recently LLMs that are favored in RL. At the same time, the ideas of learning from data are not part of the control theory tradition, and they have only been addressed relatively recently in a systematic way. 

Support the trend of increased use of ML methods in MPC. At their foundation, MPC and RL share important principles suggests that this trend will continue and accelerate in the future. 
